## Setup

In [ ]:
from specific import *

### Retrieve previous results from the 'model' notebook

In [ ]:
X_train, X_test, y_train, y_test = data_split_cache.load()
rf = get_model()

In [ ]:
X_train.shape[0]

In [ ]:
X_train.shape[0] / 2000

### SHAP values - via PBS array jobs

#### Normal SHAP values - single thread timing

~ ? s / sample, ? GB mem

In [ ]:
%%time

# get_shap_values(rf, X_train[0:100], interaction=False)

In [ ]:
plt.plot([10, 30, 50, 100], [19.6, 57.2, 95, (3 * 60) + 18], marker="o")
plt.xlabel("Samples")
plt.ylabel("time (s)")

### SHAP values - see PBS (array) job folder 'shap'!

#### Load the SHAP values (keeping track of missing chunks)

In [ ]:
max_index = 655  # Maximum job array index (inclusive).
job_samples = 2000  # Samples per job.
total_samples = (max_index + 1) * job_samples  # Sanity check.

In [ ]:
# Load the individual data chunks.
shap_chunks = []
missing = []
for index in tqdm(range(max_index + 1), desc="Loading chunks"):
    try:
        shap_chunks.append(
            SimpleCache(
                f"tree_path_dependent_shap_{index}_{job_samples}",
                cache_dir=os.path.join(CACHE_DIR, "shap"),
                verbose=0,
            ).load()
        )
    except NoCachedDataError:
        missing.append(index)

if missing:
    print("missing:", missing)
    print("nr missing:", len(missing))

shap_values = np.vstack(shap_chunks)

### Manual recalculation

In [ ]:
# import multiprocessing

# def cached_get_shap_values(index, model, X):
#     tree_path_dependent_shap_cache = SimpleCache(
#         f"tree_path_dependent_shap_{index}_{job_samples}",
#         cache_dir=os.path.join(CACHE_DIR, "shap"),
#     )
#     @tree_path_dependent_shap_cache
#     def _shap_vals():
#         return get_shap_values(model, X, interaction=False)
#     return _shap_vals()

# with multiprocessing.Pool(processes=get_ncpus()) as pool:
#     async_results = [
#         pool.apply_async(
#             cached_get_shap_values, (
#                 index,
#                 rf,
#                 X_train[index * job_samples : (index + 1) * job_samples]
#             )
#         )
#         for index in [2, 11, 12, 13, 121, 122, 123, 124, 125, 126, 127, 226, 227, 229, 231]
#     ]
#     results = [async_result.get() for async_result in async_results]

In [ ]:
%%time
with figure_saver("SHAP"):
    shap.summary_plot(
        shap_values,
        shorten_columns(X_train[:total_samples]),
        title="SHAP Feature Importances",
        show=False,
    )

In [ ]:
mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
mean_shap_importances = pd.DataFrame(
    [shorten_features(X_train.columns), mean_abs_shap], index=["column", "shap"]
)
mean_shap_importances = mean_shap_importances.T
mean_shap_importances.sort_values("shap", ascending=False, inplace=True)
mean_shap_importances

### Interaction SHAP values - see PBS (array) job folder 'shap_interaction'!

~ ? s / sample !!, ? GB mem

In [ ]:
%%time

# get_shap_values(rf, X_train[0:5], interaction=True)

In [ ]:
max_index = 5999  # Maximum job array index (inclusive).
job_samples = 50  # Samples per job.
total_interact_samples = (max_index + 1) * job_samples

shap_interact_cache = SimpleCache(
    "shap_interact_cache", cache_dir=CACHE_DIR / Path("shap_interaction")
)


@shap_interact_cache
def load_shap_interact_from_chunks():
    # Load the individual data chunks.
    shap_interact_chunks = []
    missing = []
    for index in tqdm(range(0, max_index + 1), desc="Loading chunks"):
        try:
            shap_interact_chunks.append(
                SimpleCache(
                    f"tree_path_dependent_shap_interact_{index}_{job_samples}",
                    cache_dir=os.path.join(CACHE_DIR, "shap_interaction"),
                    verbose=0,
                ).load()
            )
        except NoCachedDataError:
            missing.append(index)

    if missing:
        print("missing:", missing)
        print("nr missing:", len(missing))
        raise Exception("missing data")

    return np.vstack(shap_interact_chunks)


shap_interact_values = load_shap_interact_from_chunks()

In [ ]:
with figure_saver("SHAP_interaction"):
    shap.summary_plot(
        shap_interact_values,
        shorten_columns(X_train[:total_interact_samples]),
        title="SHAP Feature Interactions",
        show=False,
    )

In [ ]:
with figure_saver("SHAP_interaction_compact"):
    shap.summary_plot(
        shap_interact_values,
        shorten_columns(X_train[:total_interact_samples]),
        title="SHAP Feature Interactions",
        show=False,
        plot_type="compact_dot",
    )

In [ ]:
mean_interact = np.mean(np.abs(shap_interact_values), axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax = sns.heatmap(
    np.log(mean_interact),
    square=True,
    xticklabels=shorten_features(X_train.columns),
    yticklabels=shorten_features(X_train.columns),
    ax=ax,
)
ax.xaxis.set_tick_params(rotation=90)
ax.yaxis.set_tick_params(rotation=0)
_ = fig.suptitle("log(SHAP Interaction Values)")

### Get the most significant interactions

In [ ]:
def get_highest_interact_index(index, mean_interact):
    indices = np.argsort(-mean_interact[index])
    return indices[indices != index][0]

In [ ]:
masked_mean_interact = mean_interact.copy()
masked_mean_interact[np.triu_indices(mean_interact.shape[0])] = np.nan

fig, ax = plt.subplots(figsize=(10, 10))
ax = sns.heatmap(
    np.log(masked_mean_interact),
    square=True,
    xticklabels=shorten_features(X_train.columns),
    yticklabels=shorten_features(X_train.columns),
    ax=ax,
)
ax.xaxis.set_tick_params(rotation=90)
ax.yaxis.set_tick_params(rotation=0)
_ = fig.suptitle("log(SHAP Interaction Values)")

In [ ]:
interact_indices = np.tril_indices(mean_interact.shape[0])
interact_values = mean_interact[interact_indices]

interact_data = {}
for i, j, interact_value in zip(*interact_indices, interact_values):
    if i == j:
        continue
    interact_data[
        tuple(
            sorted(
                (
                    shorten_features(X_train.columns[i]),
                    shorten_features(X_train.columns[j]),
                )
            )
        )
    ] = interact_value

interact_data = pd.Series(interact_data).sort_values(ascending=False, inplace=False)

interact_data_cache = SimpleCache("SHAP_interact_data", cache_dir=CACHE_DIR)


@interact_data_cache
def dummy_func():
    return interact_data


dummy_func()

print(interact_data[:40].to_latex())

In [ ]:
interact_data[:20]

In [ ]:
interact_data[-10:]

### Compare the approximate and 'exact' interactions

In [ ]:
length = 18
header = f"{'Feature':>{length}} : {'Approx':>{length}} {'Non Approx':>{length}}"
print(header)
print("-" * len(header))

for i in range(len(X_train.columns)):
    approx = shap.common.approximate_interactions(
        X_train.columns[i], shap_values, X_train[:total_samples]
    )[0]
    non_approx = get_highest_interact_index(i, mean_interact)
    features = shorten_features(
        X_train.columns[index] for index in (i, approx, non_approx)
    )
    print(f"{features[0]:>{length}} : {features[1]:>{length}} {features[2]:>{length}}")

### Dependence plots with the approximate dependence metric

In [ ]:
def approx_dependence_plot(i, shap_values=shap_values, X=X_train[:total_samples]):
    fig, ax = plt.subplots(figsize=(7, 5))
    shap.dependence_plot(
        i, shap_values, X, alpha=0.1, ax=ax,
    )
    figure_saver.save_figure(
        fig,
        f"shap_dependence_{X_train.columns[i]}_approx",
        sub_directory="shap_dependence_approx",
    )


with concurrent.futures.ProcessPoolExecutor(max_workers=32) as executor:
    plot_fs = []
    for i in range(len(X_train.columns)):
        plot_fs.append(executor.submit(approx_dependence_plot, i))

    for plot_f in tqdm(
        concurrent.futures.as_completed(plot_fs),
        total=len(plot_fs),
        desc="Plotting approx SHAP dependence plots",
    ):
        pass

In [ ]:
def dependence_plot(
    i, shap_values=shap_values, X=X_train[:total_samples], mean_interact=mean_interact
):
    fig, ax = plt.subplots(figsize=(7, 5))
    shap.dependence_plot(
        i,
        shap_values,
        X,
        interaction_index=get_highest_interact_index(i, mean_interact),
        alpha=0.1,
        ax=ax,
    )
    figure_saver.save_figure(
        fig, f"shap_dependence_{X_train.columns[i]}", sub_directory="shap_dependence"
    )


with concurrent.futures.ProcessPoolExecutor(max_workers=32) as executor:
    plot_fs = []
    for i in range(len(X_train.columns)):
        plot_fs.append(executor.submit(dependence_plot, i))

    for plot_f in tqdm(
        concurrent.futures.as_completed(plot_fs),
        total=len(plot_fs),
        desc="Plotting SHAP dependence plots",
    ):
        pass

### SHAP force plot

#### force_plot memory usage

In [ ]:
(5e4 * (5e4 - 1) // 2) * 8 / 1e9

In [ ]:
shap.initjs()

In [ ]:
rf.n_jobs = 32
N = int(1e3)
shap.force_plot(
    np.mean(rf.predict(X_train[:N])), shap_values[:N], shorten_columns(X_train[:N])
)